## Testes da API

Nesse notebook são feitos alguns testes com a API do Olho Vivo

In [1]:
import pandas as pd
from requests import Session
from typing import Optional

In [2]:
class RequestMaker:

    def __init__(self, method:str, host:str, version:Optional[str]=None)->None:

        self.method = method
        self.host = host
        self.version = version
        self.session = Session()

        self.url_base = self.build_base_url()

    def build_base_url(self):

        url = f'{self.method}://{self.host}'

        if self.version:
            url = url+f'/v{self.version}'

        return url

    def build_params_str(self, params:dict)->str:

        params = [f"{key}={value}" for key, value in params.items()]
        params = '&'.join(params)
        params = f'?{params}'

        return params

    def build_request_url(self, endpoint:Optional[str]=None, params:Optional[dict]=None)->str:

        url = self.url_base
        if endpoint:
            url = self.url_base + '/' + endpoint.strip('/')
        
        if params:
            params_str = self.build_params_str(params)
            url = url + params_str
        
        return url
               
    def post(self,endpoint:Optional[str]=None, params:Optional[dict]=None, data:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo post em: {url}. Dados: {data}')
        with self.session.post(url, data=data) as r:
            if as_json:
                return r.json()
            return r.text
        
    def get(self, endpoint:Optional[str]=None, params:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo get em {url}.')
        with self.session.get(url) as r:
            if as_json:
                return r.json()
            return r.text



class APIOlhoVivoClient:

    HOST='api.olhovivo.sptrans.com.br'
    METHOD='http'
    VERSION='2.1'

    def __init__(self, token:str):

        self.request = RequestMaker(self.METHOD, self.HOST, self.VERSION)
        self.token=token

        self.autenticar()

    def autenticar(self)->None:

        endpoint = 'Login/Autenticar'
        params = {'token' : self.token}

        #a api nao retorna um json valido
        resp = self.request.post(endpoint, params, as_json=False)
        success = resp=='true'
        if not success:
            raise RuntimeError(f'Falha ao atenticar: {resp}')
        print('Autenticado com sucesso.')

    @property
    def posicao_atual_onibus(self):

        endpoint = 'Posicao'
        return self.request.get(endpoint)

In [3]:
token = '7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0'

In [4]:
api = APIOlhoVivoClient(token)

Fazendo post em: http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0. Dados: None
Autenticado com sucesso.


In [5]:
posicoes = []

#espera = 30
for i in range(1):
    posicao_atual = api.posicao_atual_onibus
    posicoes.append(posicao_atual)
    #time.sleep(espera)

posicoes

Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.


[{'hr': '10:06',
  'l': [{'c': '978A-10',
    'cl': 34137,
    'sl': 2,
    'lt0': 'METRÔ BARRA FUNDA',
    'lt1': 'TERM. CACHOEIRINHA',
    'qv': 8,
    'vs': [{'p': 12475,
      'a': True,
      'ta': '2025-09-17T13:05:27Z',
      'py': -23.475101000000002,
      'px': -46.670268,
      'sv': None,
      'is': None},
     {'p': 12478,
      'a': True,
      'ta': '2025-09-17T13:06:00Z',
      'py': -23.525090499999997,
      'px': -46.6665245,
      'sv': None,
      'is': None},
     {'p': 12467,
      'a': True,
      'ta': '2025-09-17T13:05:56Z',
      'py': -23.518386999999997,
      'px': -46.677789,
      'sv': None,
      'is': None},
     {'p': 12596,
      'a': True,
      'ta': '2025-09-17T13:05:41Z',
      'py': -23.4780115,
      'px': -46.672883,
      'sv': None,
      'is': None},
     {'p': 12473,
      'a': True,
      'ta': '2025-09-17T13:06:01Z',
      'py': -23.4753495,
      'px': -46.670524,
      'sv': None,
      'is': None},
     {'p': 12476,
      'a': True,

In [6]:
len(posicoes)

1

In [7]:
rows = []

for h in posicoes:
    if h:
        for l in h['l']:

            for veiculo in l['vs']:

                row = {
                    'linha': l['c'],
                    'id_onibus': veiculo['p'],
                    'timestamp': veiculo['ta'],
                    'lat': veiculo['py'],
                    'lon': veiculo['px'] 
                }

                rows.append(row)

df_posicoes = pd.DataFrame(rows)

Conversão de horário

In [8]:
df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,978A-10,12475,2025-09-17T13:05:27Z,-23.475101,-46.670268
1,978A-10,12478,2025-09-17T13:06:00Z,-23.525090,-46.666525
2,978A-10,12467,2025-09-17T13:05:56Z,-23.518387,-46.677789
3,978A-10,12596,2025-09-17T13:05:41Z,-23.478012,-46.672883
4,978A-10,12473,2025-09-17T13:06:01Z,-23.475350,-46.670524
...,...,...,...,...,...
10805,476L-10,64845,2025-09-17T13:05:48Z,-23.599649,-46.637840
10806,7411-10,82575,2025-09-17T13:06:13Z,-23.571223,-46.691788
10807,8047-41,10230,2025-09-17T13:02:14Z,-23.537255,-46.722138
10808,6058-10,66104,2025-09-17T13:06:10Z,-23.763518,-46.696115


In [9]:
df_posicoes['timestamp'] = pd.to_datetime(df_posicoes['timestamp'])

df_posicoes['timestamp'] = df_posicoes['timestamp'].dt.tz_convert("America/Sao_Paulo")

df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,978A-10,12475,2025-09-17 10:05:27-03:00,-23.475101,-46.670268
1,978A-10,12478,2025-09-17 10:06:00-03:00,-23.525090,-46.666525
2,978A-10,12467,2025-09-17 10:05:56-03:00,-23.518387,-46.677789
3,978A-10,12596,2025-09-17 10:05:41-03:00,-23.478012,-46.672883
4,978A-10,12473,2025-09-17 10:06:01-03:00,-23.475350,-46.670524
...,...,...,...,...,...
10805,476L-10,64845,2025-09-17 10:05:48-03:00,-23.599649,-46.637840
10806,7411-10,82575,2025-09-17 10:06:13-03:00,-23.571223,-46.691788
10807,8047-41,10230,2025-09-17 10:02:14-03:00,-23.537255,-46.722138
10808,6058-10,66104,2025-09-17 10:06:10-03:00,-23.763518,-46.696115


In [10]:
horas = df_posicoes["timestamp"].dt.hour

horas_unicas = horas.unique()
horas_unicas.sort()

print(horas_unicas)

[ 9 10]


Ônibus duplicados

In [13]:
duplicados = (df_posicoes.groupby(["id_onibus", "timestamp"]).filter(lambda x: len(x) > 1).sort_values(["id_onibus", "timestamp"]))

duplicados

,linha,id_onibus,timestamp,lat,lon
